In [ ]:
import pandas as pd
import re

from itertools import permutations

In [ ]:
r = re.compile('(-*|.*\s)([0-9]+\.[0-9]+|-)(.*)')

def parseCrossYearEntries(inFile):
    matches = {}
    with open(inFile, 'r') as oFile:
        for line in oFile:
            m = r.match(line)
            
            if m:
                parsed = [x.strip() for x in m.groups()]
            
                if not (parsed[0].startswith('*') or parsed[0].startswith('x') or parsed[0].startswith('--')):
                
                    if parsed[1] == '-':
                        entry_name = parsed[0]
                        parsed[1] = 0.0
                        entries = []
                    parsed[1] = float(parsed[1])
                    entries.append(parsed)

                elif parsed[0].startswith('--'):
                    if len(entries) > 1:
                        matches[entry_name] = entries
    
    return matches

In [ ]:
inFile = '../output/question-group-validation-motherSurvey-final.txt'
motherSurvey_matches = parseCrossYearEntries(inFile)
inFile = '../output/question-group-validation-fatherSurvey-final.txt'
fatherSurvey_matches = parseCrossYearEntries(inFile)

In [ ]:
def getPairMatches():
    pairMatches = pd.DataFrame()

    for k,vals in fatherSurvey_matches.iteritems():

        df = pd.DataFrame([{'key': v[0], 'score': v[1], 'desc': v[2]} for v in vals])
        df = df.set_index('key')

        perms = []
        for t,f in permutations(df.index,2):
            perms.append({
                'to': t,
                'from': f,
                'score': abs(df.loc[t].score - df.loc[f].score),
                'desc': df.loc[t].desc,
            })

        pairMatches = pd.concat([pairMatches,pd.DataFrame(perms)])

    pairMatches = pairMatches.set_index(['from', 'to'])